In [2]:
import os
import string
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
plt.style.use('sm')

/home/cody/python/lib/python2.7/site-packages/matplotlib/style/core.py:53: UserWarning: Style includes a parameter, 'toolbar', that is not related to style.  Ignoring
  "to style.  Ignoring".format(key))


In [3]:
def specplot(x1ds,lines):
    # If len(hdulist) > 2, multiple iterations of the same exposure were taken.
    # These have slightly different wavelength values, so to sum them correctly requires iterpolation onto the same
    # wavelength grid.
    # For now, just take the first of these iterations.
    #if len(hdulist)>2: 
    #    for ext in hdulist[2:]:
    #        flx=flx+ext.data[order]['flux'][4:-7]

    order=-1
    wav=[]
    flx=[]
    vel=[]
    for line in lines:
        for x1d in x1ds:
            hdulist=fits.open(x1d)
            minwave=hdulist[0].header['minwave']
            maxwave=hdulist[0].header['maxwave']
            scidata=hdulist[1].data
            if (minwave < line < maxwave):
                for i in range(len(scidata)):
                    if (scidata[i]['wavelength'][0] < line < scidata[i]['wavelength'][-1]):
                        order=i
                if order>=0:
                    wav=scidata[order]['wavelength'][4:-7]
                    vel=c*(wav-line)/line
                    flx=scidata[order]['flux'][4:-7]*float('1e12')
                    return line,wav,vel,flx
    return line,wav,vel,flx

In [7]:
path='/DataDisk/datafiles/PGCC_HST/x1d_files/'
c=float('3e5')
c_lines=[1280.1353,1328.833,1656.9283]
cl_lines=[1347.2396]
o_lines=[1355.5977]
co_lines=[1544.448,1509.748,1477.565,1447.352,1419.044,1392.525,1367.623,1344.186,1322.15,1301.403,1281.866,1263.433,1246.059]
sightlines=[x for x in os.listdir(path) if (os.path.isdir(path+x+'/E140H') and x.startswith('.')==False)]
al=string.maketrans('','')
nodigs=al.translate(al,string.digits)
sightlines.sort(key=lambda x: (len(x), int(x.translate(al,nodigs))))
#sightlines.sort(key=lambda x: len([path+x+'/E140H/'+y for y in os.listdir(path+x+'/E140H') if y.endswith('.fits')]))


In [8]:
pdf=PdfPages('spectral_comparison2.pdf')
for sightline in sightlines:

    fig,(subplt1,subplt2,subplt3,subplt4)=plt.subplots(4,1,sharex=True)
    fig.set_size_inches(15,10)
    fig.text(0.05,0.5,'Flux ($\\times 10^{-12}$ ergs (cm$^{2}$ s \\AA)$^{-1}$)',rotation=90,fontsize=25,verticalalignment='center')
    subplt4.set_xlabel('Heliocentric Velocity (km s$^{-1}$)')
    subplt1.text(1.0,1.02,sightline,transform=subplt1.transAxes,fontsize=30,horizontalalignment='right')
    
    x1ds=[path+sightline+'/E140H/'+x for x in os.listdir(path+sightline+'/E140H') if x.endswith('.fits')]
    x1ds.sort(key=lambda x: fits.open(x)[0].header['maxwave'])
        
        
    c_line,wav,vel,flx=specplot(x1ds,c_lines)
    subplt1.text(1.01,0.5, 'C I\n'+str(int(c_line)),transform=subplt1.transAxes,fontsize=30,verticalalignment='center')
    if len(flx)>0:
        subplt1.plot(vel,flx,'k-',lw=1)
        subplt1.set_ylim(-0.05*max(flx),1.1*max(flx))
        subplt1.set_xlim(-200,200)
        subplt1.xaxis.set_ticks_position('none')
        subplt1.axhline(0,0,1, linestyle='--')
        

    cl_line,wav,vel,flx=specplot(x1ds,cl_lines)
    subplt2.text(1.01,0.5, 'Cl I\n'+str(int(cl_line)),transform=subplt2.transAxes,fontsize=30,verticalalignment='center')
    if len(flx)>0:
        subplt2.plot(vel,flx,'k-',lw=1)
        subplt2.set_ylim(-0.05*max(flx),1.1*max(flx))
        subplt2.set_xlim(-200,200)
        subplt2.xaxis.set_ticks_position('none')
        subplt2.axhline(0,0,1, linestyle='--')
        
        
    o_line,wav,vel,flx=specplot(x1ds,o_lines)
    subplt3.text(1.01,0.5, 'O I\n'+str(int(o_line)),transform=subplt3.transAxes,fontsize=30,verticalalignment='center')
    if len(flx)>0:
        subplt3.plot(vel,flx,'k-',lw=1)
        subplt3.set_ylim(-0.05*max(flx),1.1*max(flx))
        subplt3.set_xlim(-200,200)
        subplt3.xaxis.set_ticks_position('none')
        subplt3.axhline(0,0,1, linestyle='--')
    
        
    x1ds.reverse()
    co_line,wav,vel,flx=specplot(x1ds,co_lines)
    subplt4.text(1.01,0.5, 'CO\n'+str(int(co_line)),transform=subplt4.transAxes,fontsize=30,verticalalignment='center')
    if len(flx)>0:
        subplt4.plot(vel,flx,'k-',lw=1)
        subplt4.set_ylim(-0.05*max(flx),1.1*max(flx))
        subplt4.set_xlim(-200,200)
        subplt4.axhline(0,0,1, linestyle='--')
        
            
    pdf.savefig()
    plt.clf()
    plt.close()
pdf.close()